# Demo: Analyzing Faces With the Azure Face Service



## Install the Azure Cognitive Services Face Library

To access the Face service from this Python notebook, you need to install the Azure Cognitive Services Face Library. This library is part of the [Azure SDK for Python](https://github.com/Azure/azure-sdk-for-python) GitHub project.


In [ ]:
pip install azure-cognitiveservices-vision-face

Import the libraries and reference the services required to execute the cells below.

In [ ]:
# Import the libraries need to call the Face service and execute the remaining cells in this notebook
from azure.cognitiveservices.vision.face import FaceClient
from msrest.authentication import CognitiveServicesCredentials
from py_code import faces
import matplotlib.pyplot as plt
from PIL import Image
import os

## Set variables

Enter the key and endpoint of your Face Service resource. To use your cognitive services resource, client applications need its endpoint and authentication key.

In [ ]:
key = '008bc0c83d6840b3badbf662f040925d' #'YOUR_COGNITIVE_SERVICES_KEY'
endpoint = 'https://udacity-cog-services.cognitiveservices.azure.com/' #'YOUR_COGNITIVE_SERVICES_ENDPOINT'

print('Ready to analyze faces using the Azure Face service at {}.'.format(endpoint))

## Create a Face client

After setting the `key` and `endpoint` values needed to access your Face service, you can instantiate a new client.

Execute the cell below to create a `FaceClient` object.

In [ ]:
# Instantiate a Face client
client = FaceClient(endpoint, CognitiveServicesCredentials(key))

## Detect faces

The `detect_with_stream()` method of the Python SDK allows us to pass in stream data for an image and get the resulting information back from the Face service.

In [ ]:
%matplotlib inline

# Set the path to the image
imagePath = os.path.join('test-images', 'women.jpeg')

# Detect faces
with open(imagePath, mode="rb") as imageData:
    faceResults = client.face.detect_with_stream(imageData)

# Display the faces
faces.show_faces(imagePath, faceResults)

Each detected face is assigned a unique ID, allowing your application to identify each individual face that was detected.

Execute the cell below to see the ID for the face of the woman detected in the previous cell.

In [ ]:
# Set the path to the image
imagePath = os.path.join('test-images', 'woman.jpeg')

# Detect faces
with open(imagePath, mode="rb") as imageData:
    faceResults = client.face.detect_with_stream(imageData)
    
# Display the faces
faces.show_faces(imagePath, faceResults, show_id=True)

## Analyze facial attributes

The Azure Face service is capable of much more than simply detect faces. Using the `return_face_attributes` argument, we can also analyze facial features and expressions. These attributes allow the Face service to predict approximate age and evaluate any emotional expressions present on the face.

Execute the cell below to analyze the facial attributes of the woman in our image.

In [ ]:
# Open an image
imagePath = os.path.join('test-images', 'women.jpeg')

# Detect faces and specified facial attributes
attributes = ['age', 'gender', 'emotion']

# Detect faces
with open(imagePath, mode="rb") as imageData:
    faceResults = client.face.detect_with_stream(imageData, return_face_attributes=attributes)

# Display the faces
faces.show_faces(imagePath, faceResults, show_attributes=True)

## Find similar faces

When faces are detected using the Face service, each one is assigned a unique `Face Id`. Face Ids are used to individually identify face detections. You can use these Ifs to compare a detected face to previously detected faces and find faces with similar features.

Run the cell below to compare the man in one image with the man in another, and find a matching face.

In [ ]:
from importlib import reload
faces = reload(faces)

In [ ]:
# Set the paths to the two images to compare
imageOnePath = os.path.join('test-images', 'man-similar-01.jpeg')
imageTwoPath = os.path.join('test-images', 'man-similar-02.jpeg')

# Detect faces in the first image
with open(imageOnePath, mode="rb") as imageOneData:
    faceOneResults = client.face.detect_with_stream(imageOneData)

# Retrieve the first face identified in the image
faceOne = faceOneResults[0]

# Detect faces in the second image
with open(imageTwoPath, mode="rb") as imageTwoData:
    faceTwoResults = client.face.detect_with_stream(imageTwoData)

# Retrieve the face Ids found in the second image.
imageTwoFaceIds = list(map(lambda face: face.face_id, faceTwoResults))

# Find faces in image two that are similar to the face in image one
similarFaces = client.face.find_similar(face_id=faceOne.face_id, face_ids=imageTwoFaceIds)

# Show the face in image 1, and similar faces in image 2
faces.show_similar_faces(imageOnePath, faceOne, imageTwoPath, faceTwoResults, similarFaces)

## Recognize faces

We've demonstrated that the Azure Face service can detect faces and analyze facial attributes, and that is is capable of identifying similar faces.

Now, let's take a look at what is involved in implementing a facial recognition solution using the Python SDK.

in which you train Face to recognize a specific person's face. This can be useful in a variety of scenarios, such as automatically tagging photographs of friends in a social media application, or using facial recognition as part of a biometric identity verification system.

To see how this works, let's use facial recognition to identify a friend named Travis.

The first step in this process is to create a person group to store our friends.

In [ ]:
# Provide a group Id
groupId = 'friends_group'
try:
    # Delete group if it already exists
    client.person_group.delete(groupId)
except Exception as ex:
    print(ex.message)
finally:
    client.person_group.create(groupId, 'friends')
    print ('Group created!')

Now that the person group exists, we can add a person for each employee we want to include in the group, and then register multiple photographs of each person so that Face can learn the distinct facial characetristics of each person. Ideally, the images should show the same person in different poses and with different facial expressions.

We'll add a single employee called Wendell, and register three photographs of the employee.

In [ ]:
%matplotlib inline

# Add a person (Travis) to the group
travis = client.person_group_person.create(groupId, 'Travis')

# Get photo's of Travis
folder = os.path.join('test-images', 'travis')
travisPics = os.listdir(folder)

# Register the photos
i = 0
fig = plt.figure(figsize=(8, 8))
for pic in travisPics:
    # Add each photo to person in person group
    if pic.endswith('.jpeg'):
        imagePath = os.path.join(folder, pic)
        
        # Add the face detected in the image to the person group
        with open(imagePath, mode="rb") as imageData:
            client.person_group_person.add_face_from_stream(groupId, travis.person_id, imageData)

        # Display each image
        img = Image.open(imagePath)
        i+=1
        a=fig.add_subplot(1,len(travisPics), i)
        a.axis('off')
        imgplot = plt.imshow(img)
plt.show()

With the person added and their photographs registered, we can now train the Face service to recognize the person.

In [ ]:
client.person_group.train(groupId)
print('Trained!')

Now, we can use the model trained to recognize faces in an image and return the person's identity.

In [ ]:
# Get the face IDs in an image
imagePath = os.path.join('test-images', 'friend-04.jpeg')

# Detect faces
with open(imagePath, mode="rb") as imageData:
    faceResults = client.face.detect_with_stream(imageData)
    
imageFaceIds = list(map(lambda face: face.face_id, faceResults))

# Get recognized face names
faceNames = {}
recognizedFaces = client.face.identify(imageFaceIds, groupId)
for face in recognizedFaces:
    personName = client.person_group_person.get(groupId, face.candidates[0].person_id).name
    faceNames[face.face_id] = personName

# show recognized faces
faces.show_recognized_faces(imagePath, faceResults, faceNames)